In [ ]:
# Using tensorflow python 37 Environment

In [1]:
import logging
from typing import Optional, Tuple

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# from PyEMD.splines import akima, cubic_spline_3pts
# from PyEMD.utils import get_timeline

import pylab as plt
import warnings
import time
from pandas import concat
from pandas import DataFrame
from numpy import array

In [2]:
from math import sqrt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation, Dropout, Flatten, Conv1D, MaxPooling1D, ConvLSTM2D
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

In [ ]:
# 01. Upload Data

In [3]:
df_Br = pd.read_csv('Br_close.csv')
df_Br.set_index(pd.DatetimeIndex(df_Br['Date']), inplace=True)
df_Br.drop(['Date'],axis=1,inplace=True)
df_Br.head()

,Close
Date,
2000-01-04,24.39
2000-01-05,23.73
2000-01-06,23.62
2000-01-07,23.09
2000-01-10,23.73


In [ ]:
# 02. Knowing How Much IMF in each Date

In [ ]:
# Upload Data Jumlah Kol

In [4]:
df_Kol = pd.read_csv('Jumlah_IMF_CEEMD.csv')
df_Kol.set_index(pd.DatetimeIndex(df_Kol['Date']), inplace=True)
df_Kol.drop(['Date'],axis=1,inplace=True)
df_Kol.head()

,Jum_Kol
Date,
2018-11-27,10
2018-11-28,11
2018-11-29,10
2018-11-30,11
2018-12-03,11


In [ ]:
# 03. Univariate MLP

In [ ]:
# Prediksi data IMF 1 semua

In [5]:
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [6]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [7]:
def series_to_supervised(data, n_in, n_out=60):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

In [8]:
# Untuk Horizon lebih dari satu
def split_sequence2(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Testing 2 Hor :

In [9]:
lim = 840
iIMF = 1 # Letak IMF yang diambil
n_test = 60 # horizon = 35
pred_iIMF = []

In [10]:
pred_all = []

In [12]:
# Parameter and Define MLP Model 
n_steps = 80 # sebelumnya 36
n_input = 80
n_steps_out = 60
n_nodes = 150
n_epochs = 100
n_batch = 100

In [ ]:
# Ini Proses Sekaligus semua IMF
start = time.time()
lim = 840
n_test = 60 # horizon = 2

pred = []

for i in range (1, 841): # s/d 901
    if i % 60 != 0:
        continue
    df_coba2 = pd.read_csv(f'CEEMD_-{lim-i}.csv')
    df_coba2.set_index(pd.DatetimeIndex(df_coba2['Date']), inplace=True)
    df_coba2.drop(['Date'],axis=1,inplace=True)
    bar, kol = df_coba2.shape
    all_pred = np.zeros(60) # dua elemen karena horizon 2
    
    for j in range (0, kol):
        imfs_x = df_coba2[f'{j+1}'].values
        train, test = train_test_split(imfs_x, n_test)
        
        # prepare data
        data = series_to_supervised(train, n_input)
        train_x, train_y = data[:, :-60], data[:, -60:] # dikasih -30:
        # define model
        model = Sequential()
        model.add(Dense(n_nodes, activation='relu', input_dim=n_input))
        model.add(Dense(60))
        model.compile(loss='mse', optimizer='adam')
        # fit
        model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
        
        # history = [x for x in train]
        x_input = train[-n_steps:].reshape((1, n_steps))
        # x_input = array(history[-n_input:]).reshape(1, n_input)
        # forecast
        yhat = model.predict(x_input, verbose=0)
    
        for k in range(0, n_steps_out):
            all_pred[k] += list(yhat[0])[k]
                   
    for l in range(0, n_steps_out):
        pred.append(all_pred[l])
    print(' > proses ke %i' % i)

print('it took: ', (time.time()-start)*1000)
pred

 > proses ke 480
 > proses ke 510
 > proses ke 540
 > proses ke 570
 > proses ke 600
 > proses ke 630
 > proses ke 660


In [ ]:
# Ini Proses tiap IMF

In [ ]:
# INGAT NEXT adalah
# Pengambilan s/d 121 - 151 ; len harusnya 5, 10 atau 15; total sampai 246 - 251, s/d Proses 50
# Proses 24 ke 30 
'''for k in range(0, len(pred_iIMF)):
    pred_all.append(pred_iIMF[k])
len(pred_all)'''

In [ ]:
# pred_all

In [ ]:
# pred_all

In [ ]:
# Simpan File tiap 50
'''
lim = 250
i = 10
df_coba3 = pd.DataFrame(df_Br.index[-250:-240].values, columns=['Date'])
df_coba3['pred'] = pred_all
df_coba3.set_index(pd.DatetimeIndex(df_coba3['Date']), inplace=True)
df_coba3.drop(['Date'],axis=1,inplace=True)'''
# df_coba3.to_csv(f'Pred_MLP1_iINF1_-{lim}-{lim-i}.csv',index=True)
# df_coba3.to_excel(f'Pred_MLP1_iINF1_-{lim}-{lim-i}.xlsx',sheet_name='Sheet1',index=True)

In [ ]:
# Ini Yang sekaligus IMF

In [14]:
# INGAT NEXT adalah
# Pengambilan s/d 121 - 151 ; len harusnya 5, 10 atau 15; total sampai 246 - 251, s/d Proses 50
# Proses 24 ke 30 
for k in range(0, len(pred)):
    pred_all.append(pred[k])
len(pred_all)

450

In [15]:
pred_all

[52.776236809790134,
 53.76026549190283,
 54.732227981090546,
 53.583981558680534,
 53.26768797636032,
 53.70676676928997,
 54.709977181628346,
 55.176204800605774,
 57.06734611093998,
 57.41762122511864,
 57.87475875765085,
 58.151406824588776,
 60.404584899544716,
 60.5330153927207,
 60.87082353234291,
 61.52160701298271,
 62.34575375914574,
 61.126445919275284,
 61.25374299287796,
 60.35468965768814,
 61.080808877944946,
 60.98763858526945,
 61.066810762509704,
 60.32539162039757,
 60.87377391755581,
 59.77375113964081,
 61.342189729213715,
 60.68982197344303,
 60.440769001841545,
 59.171523317694664,
 62.70017406344414,
 62.56637728214264,
 61.48631927371025,
 61.30392390489578,
 61.19397822022438,
 62.23017102479935,
 63.80173260718584,
 65.60272427275777,
 65.55296387523413,
 65.59524347539991,
 65.70601547509432,
 64.90792036987841,
 65.76167775690556,
 65.93437317758799,
 66.17379296943545,
 65.83827666938305,
 65.14137632772326,
 65.19927810132504,
 64.89041761308908,
 66.1146

In [72]:
# Simpan File tiap 50
lim = 840
i = 840
df_coba3 = pd.DataFrame(df_Br.index[-840:].values, columns=['Date'])
df_coba3['pred'] = pred_all
df_coba3.set_index(pd.DatetimeIndex(df_coba3['Date']), inplace=True)
df_coba3.drop(['Date'],axis=1,inplace=True)
df_coba3.to_csv(f'Pred_MLP60_150_CEEMD-{lim}-{lim-i}.csv',index=True)
df_coba3.to_excel(f'Pred_MLP60_150_CEEMD-{lim}-{lim-i}.xlsx',sheet_name='Sheet1',index=True)

In [ ]:
# 04. Hitung Error Measurement

In [16]:
def timeseries_evaluation_metrics_func(y_true, y_pred):    
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

In [19]:
# len(df_Br['Close'][-900:-450])

450

In [20]:
timeseries_evaluation_metrics_func(df_Br['Close'][-840:], pred_all)

Evaluation metric results:-
MSE is : 21.745484610139826
MAE is : 2.9868748591494043
RMSE is : 4.6632054008096
MAPE is : 6.8922772577159686
R2 is : 0.8791442273577725

